<h1 style='color: blue; font-size: 34px; font-weight: bold;'> Projetos de Data Science
</h1>

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Este Notebook concentra a descrição do Projeto de Análise de Churn para a empresa Telco.  </i> 
</p>  

# <font color='red' style='font-size: 40px;'> Consultas </font>
<hr style='border: 2px solid red;'>

> CONSULTAR:

https://www.kaggle.com/datasets/datacertlaboratoria/projeto-5

https://www.kaggle.com/code/datacertlaboratoria/guia-de-resolu-o-projeto-5

# <font color='red' style='font-size: 40px;'> Problemática </font>
<hr style='border: 2px solid red;'>

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> TELCO é uma empresa jovem e com muito potencial, cujas filiais estão localizadas em 1.106 cidades do estado da Califórnia, Estados Unidos.  </i> 
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> A CEO da empresa queria entender como diminuir a perda de clientes na TELCO, portanto, durante as apresentações, um dos Diretores disse as seguintes hipóteses: 

    1. A perda de clientes na TELCO se deve ao aumento da concorrência. Há cada vez mais concorrentes no mercado, principalmente na região de San Diego, o que nos faz perder mais clientes lá do que em outros lugares.

    2. O problema são os contratos mensais. Se pudéssemos migrar todos os nossos clientes para contratos mais longos, todos os nossos problemas seriam resolvidos.

 </i> 
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Após ouvir as declarações do Diretor, a CEO disse: “É claro para mim que os contratos mês a mês são mais propensos à perda do que os outros, mas como sabemos se esse efeito é o mesmo para todas as idades, ou se é o mesmo para os clientes com mais ou menos antiguidade? Da mesma forma, um cliente que indicou muitos outros clientes tem menos probabilidade de sair, mas como sabemos se o efeito mês a mês é mais ou menos importante do que o número de referências de um cliente? Além disso, assumindo que a estratégia que você propõe acaba sendo a correta, ou seja, em geral buscamos movimentar os clientes para contratos mais longos, como priorizamos? Quem contatamos primeiro? Gostaria de fazer uma análise profunda da situação para entendê-la melhor.”
 </i> 
</p>  

<p style='font-size: 18px; line-height: 2; margin: 0px 0px; text-align: justify; text-indent: 0px;'>    
<i> Dado esta problemática, ficou acordado o desenvolvimento de um relatório analítico com o seguinte:

    1. Uma visão geral da população de clientes TELCO. A CEO quer entender as características dos clientes TELCO. Qual é a divisão por sexo? Eles têm dependentes? idades? cidades? etc.

    2. Uma exploração de perdas para entender melhor as suposições do Diretor sobre contratos mensais e a região de San Diego como a mais afetada. Algumas segmentações para entender a perda de clientes que saíram durante a reunião são:

    - Grupo 1: Contrato mes-a-mes e idade > 64
    - Grupo 2: Contrato mes-a-mes , idade < 64 e número de referências < 2
    - Grupo 3: Contrato diferente de mes-a-mes, edad > 64 y número de referidos < 2
    - Grupo 4: Contrato diferente a mes-a-mes e antiguidade em meses < 40

    3. Uma análise de clientes em “alto risco” de abandono priorizada de acordo com o valor monetário estimado que representam para a TELCO. Aqui a ideia é que após identificar os fatores que estão relacionados a perda de clientes (com base nos clientes que já saíram), você possa identificar quais clientes, da base de clientes que a TELCO ainda tem (ou seja, clientes que não saíram), se eles seriam mais propensos a deixar a empresa. E a partir dessa base, identifique aqueles clientes que possuem um alto valor monetário para a TELCO. Em outras palavras, você deseja priorizar aqueles clientes que são: i) propensos a sair (não saíram, mas estariam em risco de sair) e ii) de alto valor monetário. Amalia quer a lista desses clientes.
 </i> 
</p>  


# <font color='red' style='font-size: 40px;'> Bibliotecas Utilizadas </font>
<hr style='border: 2px solid red;'>

In [2]:
## Bibliotecas De Manipulação de Dados e Visualização

import pandas as pd 
from pandasql import sqldf
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px 
import plotly.graph_objects as go

## Bibliotecas de Modelagem Matemática e Estatística
import numpy as np
import scipy as sp 
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Bibliotecas de Manipulação de Tempo
import time
import datetime


# Bibliotecas de Suporte
import warnings
import pyodbc



# Parâmetros de Otimização
%matplotlib inline
sns.set()
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# <font color='red' style='font-size: 40px;'> Conexão SQL Server </font>
<hr style='border: 2px solid red;'>

In [3]:
dados_conexao = (
    'Driver={SQL Server};'
    'Server=DESKTOP-A7BL36E;'
    'Database=TELCO;'
)

conexao = pyodbc.connect(dados_conexao)
print('Conexao Bem Sucedida')

cursor = conexao.cursor()

Conexao Bem Sucedida


In [4]:
demographics = pd.read_sql('SELECT * FROM demographics', conexao)
locations = pd.read_sql('SELECT * FROM locations', conexao)
populations = pd.read_sql('SELECT * FROM populations', conexao)
services = pd.read_sql('SELECT * FROM services', conexao)
status = pd.read_sql('SELECT * FROM status', conexao)

# <font color='red' style='font-size: 40px;'> Funções </font>
<hr style='border: 2px solid red;'>

# <font color='red' style='font-size: 40px;'> 1.0 Entendimento das Bases de Dados </font>
<hr style='border: 2px solid red;'>

## 1.1) DEMOGRAPHICS

## 1.2) LOCATIONS

## 1.3) POPULATIONS

## 1.4) SERVICES

## 1.5) STATUS

# <font color='red' style='font-size: 40px;'> 2.0 Análise Exploratória </font>
<hr style='border: 2px solid red;'>

### 2.1) Uma visão geral da população de clientes TELCO visando entender as características dos clientes TELCO. 
- Qual é a divisão por sexo? 
- Eles têm dependentes? 
- E as Idades? 
- E as Cidades? 
- etc.

### 2.2) Uma exploração de perdas para entender melhor as suposições do Diretor sobre contratos mensais e a região de San Diego como a mais afetada. Algumas segmentações para entender a perda de clientes que saíram durante a reunião são:

- Grupo 1: Contrato mes-a-mes e idade > 64
- Grupo 2: Contrato mes-a-mes , idade < 64 e número de referências < 2
- Grupo 3: Contrato diferente de mes-a-mes, idade > 64 e número de referências < 2
- Grupo 4: Contrato diferente a mes-a-mes e antiguidade em meses < 40

    

### 2.3) Identificação de clientes que são mais propensos a deixarem a empresa e, destes clientes, os que possuem alta representatividade monetária para a TELCO:

- Clientes mais propensos a sair (não saíram, mas estariam em risco de sair)

- Dos Clientes mais propensos, classificar os de de alto valor monetário e mandar para a CEO

# <font color='red' style='font-size: 40px;'> 3.0 Pré-Processamento </font>
<hr style='border: 2px solid red;'>

# <font color='red' style='font-size: 40px;'> 4.0 Modelagem I </font>
<hr style='border: 2px solid red;'>

# <font color='red' style='font-size: 40px;'> 5.0 Modelagem II </font>
<hr style='border: 2px solid red;'>

# <font color='red' style='font-size: 40px;'> 6.0 Consolidação dos Resultados </font>
<hr style='border: 2px solid red;'>